In [12]:
import zarr # Check version
import numpy as np

zarr.__version__

'2.13.2'

In [4]:
ncar_data = zarr.open("sciserver-experimental/ariel/tests")

In [5]:
ncar_data.info

Name,/
Type,zarr.hierarchy.Group
Read-only,False
Store type,zarr.storage.DirectoryStore
No. members,6
No. arrays,6
No. groups,0
Arrays,"e, p, t, u, v, w"


In [6]:
ncar_data['e'].info

Name,/e
Type,zarr.core.Array
Data type,float32
Shape,"(2048, 2048, 2048)"
Chunk shape,"(256, 256, 256)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,34359738368 (32.0G)
No. bytes stored,13989514907 (13.0G)


<font color="orange">ok, the data is compressed. Let's test access time</font>

In [14]:
chunk_size = (256, 256, 256)
print("Chunk Size: ", np.product(np.array(chunk_size)) / (1024**2), " MB")

Chunk Size:  16.0  MB


In [8]:
ncar_data['e'][0,0,0]

0.03261032

In [20]:
%%timeit -o

a = ncar_data['e'][:256,:256,:256]

The slowest run took 4.92 times longer than the fastest. This could mean that an intermediate result is being cached.
376 ms ± 209 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 376 ms ± 209 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [ ]:
%%timeit -o

a = ncar_data['e'][1000:1256,1000:1256,1000:1256]